In [77]:
#PACKAGE IMPORTS

# Nice Formatting within Jupyter Notebook
%matplotlib inline
from IPython.display import display # Allows multiple displays from a single code-cell

# System functionality
import sys
sys.path.append('..')

# Modules & utilities
import os
import sys
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import itertools
import collections
import matplotlib.ticker as mtick

In [78]:
#Functions

def nz(value):

    '''
    Convert None to int zero else return value.
    '''

    if value == None:
        return 0
    return value

In [79]:


#Raw data initial exploration
#Load small dataset


small_raw = pd.read_table('orange_small_train.data').replace('\\', '/')
#display(small_raw.shape) #shows the shape of the full data set (number of rows and columns)
#display(small_raw.describe()) #shows the dataset statistics
#display(small_raw.head(5))
#print(small_raw.info())



In [80]:


#Check and remove empty columns
small=small_raw.dropna(how='all', axis=1) #drop empty columns
removed_columns = small_raw.shape[1] - small.shape[1]  
#print("Removed empty columns: " + str(removed_columns)) #confirm how many columns were removed

#Check if there are duplicate columns in the remaining dataset:
dup_count =small.columns.duplicated() #array of true/false for duplicated/non-duplicated columns
d=collections.Counter(dup_count)
#print("Number of duplicate columns:"+str(nz(d.get(True)))) #use none to zero function

#print(small.info())



In [81]:
#Show dataset statistics again:
pd.set_option('display.max_columns', 230)
#display(small.describe()) #shows the dataset statistics
#display(small.head(5))

In [82]:
#Check the number of unique entries per column:
pd.set_option('display.max_rows', 230)
#small.nunique()

In [83]:
#Isolating integer variable
for var in small.columns:
    if small[var].dtype == int:
        #print(var)
        
        
#print(small['Var73'])


SyntaxError: unexpected EOF while parsing (<ipython-input-83-fe759d9d6665>, line 7)

In [84]:
#Replacing object columns with categorical

small_copy = small.copy(deep=True)
for var in small_copy.columns:
    if small_copy[var].dtype == 'object':
        cat_col = small_copy[var].astype('category')
        small_copy.loc[:,var] = cat_col
        #print('Categories for ',var, ': ',small_copy[var].cat.categories)

In [85]:
#ax = sns.countplot(x='Var197', data=small_copy)
cat_indexes = []
for i in range(191,230,1):
    cat_indexes.append('Var'+str(i))
    
cat_indexes.remove('Var209')
    
#print(cat_indexes)

#plt.subplot(8,5, figsize=(5,5))
#plt.figure(figsize=(30,20))
#for i in range(0,38,1):
 #   plt.subplot(8,5,i+1)
 #   #plt.figure(figsize=(5,5))
 #   ax = sns.countplot(x=cat_indexes[i], data=small_copy)
 #   plt.xticks([])
 #   print('Plotted ',i)
    
#plt.show()

for i in range(0,38,1):
    #print(small_copy[cat_indexes[i]].value_counts()[:10])
    
#1 value Vars:191, 201, 213, 215, 224
#2 value Vars: 208, 211, 218
#3 value Vars:194,205
#Very skewed distriburion: 195, 196, 203, 210, 219


SyntaxError: unexpected EOF while parsing (<ipython-input-85-d711bcd84c01>, line 27)

In [86]:
# Remove columns with a certain threshold of null values
#Threshold should be a proportion of how many values needed to keep column
#i.e. if we only want columns with 70% non-null values or more, threshold should be 0.7
def removenull(data, threshold):
    datacopy = data.copy(deep=True)
    datacopy2 = datacopy.dropna(axis=1, thresh=int((datacopy.shape[0])*threshold))
    print('Removed ', data.shape[1]-datacopy2.shape[1], 'columns using threshold ', '{0:.2f}'.format(threshold))
    return datacopy2

#print(removenull(small_copy, 0.9).describe())
#print(removenull(small_copy, 0.5).describe())
#print(removenull(small_copy, 0.2).describe())

print('Original number of columns: ', small_copy.shape[1])
for i in np.arange(0,1,0.05):
    removenull(small_copy, i)


Original number of columns:  212
Removed  0 columns using threshold  0.00
Removed  135 columns using threshold  0.05
Removed  136 columns using threshold  0.10
Removed  136 columns using threshold  0.15
Removed  136 columns using threshold  0.20
Removed  136 columns using threshold  0.25
Removed  138 columns using threshold  0.30
Removed  138 columns using threshold  0.35
Removed  138 columns using threshold  0.40
Removed  140 columns using threshold  0.45
Removed  143 columns using threshold  0.50
Removed  143 columns using threshold  0.55
Removed  145 columns using threshold  0.60
Removed  145 columns using threshold  0.65
Removed  145 columns using threshold  0.70
Removed  146 columns using threshold  0.75
Removed  146 columns using threshold  0.80
Removed  146 columns using threshold  0.85
Removed  184 columns using threshold  0.90
Removed  184 columns using threshold  0.95


In [87]:
#Loads in all three targets into a separate dataframe
app_labels = pd.read_csv(os.path.join(os.getcwd(), 'orange_small_train_appetency.csv'), header=None)
churn_labels = pd.read_csv(os.path.join(os.getcwd(), 'orange_small_train_churn.csv'), header=None)
upsell_labels = pd.read_csv(os.path.join(os.getcwd(), 'orange_small_train_upselling.csv'), header=None)
small_labels = pd.concat([app_labels, churn_labels, upsell_labels], axis=1)
print(small_labels.head(10))
small_labels.columns = ['Appetency', 'Churn','Upselling']
print(small_labels.head(10))

   0  0  0
0 -1 -1 -1
1 -1  1 -1
2 -1 -1 -1
3 -1 -1 -1
4 -1 -1 -1
5 -1 -1 -1
6 -1 -1 -1
7 -1 -1 -1
8 -1 -1 -1
9 -1 -1 -1
   Appetency  Churn  Upselling
0         -1     -1         -1
1         -1      1         -1
2         -1     -1         -1
3         -1     -1         -1
4         -1     -1         -1
5         -1     -1         -1
6         -1     -1         -1
7         -1     -1         -1
8         -1     -1         -1
9         -1     -1         -1


In [88]:

# Select Float columns, create indicators of missing values, replace NaN with mean
# output to small_f

small_1= small_copy.loc[:, small_copy.dtypes == np.float64]
for col in small_1.columns:
    small_1[col+"_missing"] = small_1[col].isnull().astype(int)
    small_1[col].fillna((small_1[col].mean()), inplace=True)

#display(small_1)



/usr/lib/python3.4/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/lib64/python3.4/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [89]:
#create correlation matrix
corr_matrix = small_1.loc[:, small_1.dtypes == np.float64].corr().abs()

In [90]:
#Set a maximum cap for correlation 
corr_cap = 0.8 #max acceptable correlation

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than max acceptable
to_drop1 = [column for column in upper.columns if any(upper[column] > corr_cap)]  #float columns
to_drop2=[str(i+'_missing') for i in to_drop1] #binary missing data indicator columns
to_drop=to_drop1+to_drop2 #all columns to remove

print('Total highgly correlated columns to remove:'+str(len(to_drop1)))
print(to_drop1)

Total highgly correlated columns to remove:48
['Var9', 'Var22', 'Var25', 'Var43', 'Var60', 'Var61', 'Var62', 'Var63', 'Var66', 'Var68', 'Var71', 'Var77', 'Var83', 'Var88', 'Var89', 'Var91', 'Var96', 'Var99', 'Var101', 'Var103', 'Var104', 'Var105', 'Var107', 'Var112', 'Var115', 'Var120', 'Var121', 'Var123', 'Var127', 'Var128', 'Var129', 'Var132', 'Var145', 'Var148', 'Var151', 'Var156', 'Var157', 'Var159', 'Var160', 'Var164', 'Var166', 'Var170', 'Var174', 'Var178', 'Var179', 'Var184', 'Var186', 'Var187']


In [91]:
#Drop correlated columns, show df head and the matrix

small_2=small_1.drop(small_1[to_drop], axis=1)
c=(small_2.loc[:, small_1.dtypes == np.float64]).shape

print('Float columns kept:'+str(c[1]))
display(small_2.head(5))



#Visualise the new correlation matrix
corr_matrix2 = small_2.loc[:, small_1.dtypes == np.float64].corr().abs()
# Generate a mask for the upper triangle
mask = np.zeros_like(corr_matrix2, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
#f, ax = plt.subplots(figsize=(20, 20))

# Generate a custom diverging colormap
#cmap = sns.diverging_palette(10, 220, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
#sns.heatmap(corr_matrix2, mask=mask, cmap=cmap, vmax=1, center=0,annot=False, annot_kws={"size": 8}, 
            #square=True, linewidths=.5, cbar_kws={"shrink": .5})
#plt.show()

Float columns kept:125


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var10,Var11,Var12,Var13,Var14,Var16,Var17,Var18,Var19,Var21,Var23,Var24,Var26,Var27,Var28,Var29,Var30,Var33,Var34,Var35,Var36,Var37,Var38,Var40,Var41,Var44,Var45,Var46,Var47,Var49,Var50,Var51,Var53,Var54,Var56,Var57,Var58,Var59,Var64,Var65,Var67,Var69,Var70,Var72,Var74,Var75,Var76,Var78,Var80,Var81,Var82,Var84,Var85,Var86,Var87,Var90,Var92,Var93,Var94,Var95,Var97,Var98,Var100,Var102,Var106,Var108,Var109,Var110,Var111,Var113,Var114,Var116,Var117,Var118,Var119,Var122,Var124,Var125,Var126,Var130,Var131,Var133,Var134,Var135,Var136,Var137,Var138,Var139,Var140,Var142,Var143,Var144,Var146,Var147,Var149,Var150,Var152,Var153,Var154,Var155,Var158,Var161,Var162,Var163,Var165,Var168,Var171,Var172,...,Var13_missing,Var14_missing,Var16_missing,Var17_missing,Var18_missing,Var19_missing,Var21_missing,Var23_missing,Var24_missing,Var26_missing,Var27_missing,Var28_missing,Var29_missing,Var30_missing,Var33_missing,Var34_missing,Var35_missing,Var36_missing,Var37_missing,Var38_missing,Var40_missing,Var41_missing,Var44_missing,Var45_missing,Var46_missing,Var47_missing,Var49_missing,Var50_missing,Var51_missing,Var53_missing,Var54_missing,Var56_missing,Var57_missing,Var58_missing,Var59_missing,Var64_missing,Var65_missing,Var67_missing,Var69_missing,Var70_missing,Var72_missing,Var74_missing,Var75_missing,Var76_missing,Var78_missing,Var80_missing,Var81_missing,Var82_missing,Var84_missing,Var85_missing,Var86_missing,Var87_missing,Var90_missing,Var92_missing,Var93_missing,Var94_missing,Var95_missing,Var97_missing,Var98_missing,Var100_missing,Var102_missing,Var106_missing,Var108_missing,Var109_missing,Var110_missing,Var111_missing,Var113_missing,Var114_missing,Var116_missing,Var117_missing,Var118_missing,Var119_missing,Var122_missing,Var124_missing,Var125_missing,Var126_missing,Var130_missing,Var131_missing,Var133_missing,Var134_missing,Var135_missing,Var136_missing,Var137_missing,Var138_missing,Var139_missing,Var140_missing,Var142_missing,Var143_missing,Var144_missing,Var146_missing,Var147_missing,Var149_missing,Var150_missing,Var152_missing,Var153_missing,Var154_missing,Var155_missing,Var158_missing,Var161_missing,Var162_missing,Var163_missing,Var165_missing,Var168_missing,Var171_missing,Var172_missing,Var173_missing,Var176_missing,Var177_missing,Var180_missing,Var181_missing,Var182_missing,Var183_missing,Var188_missing,Var189_missing,Var190_missing
0,11.487179,0.004029,425.298387,0.125396,238793.32885,1526.000000,7.0,392605.656355,8.625806,16.071685,184.0,0.741935,120.239275,11.393287,7.215959,0.245092,464.000000,7.535306,14.000000,0.070612,0.028245,166.56,0.022792,7.435897,127004.950413,1.173247,0.0,159553.853344,648522.148195,3570.0,13.962933,26.653846,0.0,7256.127907,16.796132,2.188034,0.091861,35.690883,43652.006725,654326.5,4.789686,87964.667183,4.076907,164061.333333,414596.520976,26526.471067,9.0,0.016812,3.531945e+06,400340.558171,4.190659,35.0,6.498791,1350864.0,0.0,54421.039005,7333.110000,2.426219,42.527419,12.0,286892.974359,5.424501,0.019943,170679.444444,2.127774,98671.065858,109771.411765,0.915938,21295.060932,0.887464,28765.797421,38164.132362,193779.51567,104.00000,6.683761,284824.232416,117625.60,608132.609186,0.064103,129837.29449,3.0,1175.000000,0.043513,223079.148828,720.0,8.00000,0.495968,4.064877e+06,1212385.0,69134.0,199.920443,110690.905588,3.82906,0.001267,181541.37189,185.0,1.128205,0.0,9.000000,2.781439,1.701412,397579.000000,157687.615579,8.314123,1812252.0,1.538221e+06,0.80114,1.896907,3.374288,336016.762288,38418.0,28842.355097,332.938575,367451.723236,9.744452,...,0,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,1,0,0,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,0,1,1,0,0,1,1,1,1,1,0,1,0,0,1,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1,0,1,1,1,0,1,1,1,0,1
1,11.487179,0.004029,425.298387,0.125396,238793.32885,525.000000,0.0,392605.656355,8.625806,16.071685,0.0,0.741935,120.239275,11.393287,7.215959,0.245092,168.000000,7.535306,2.000000,0.070612,0.028245

In [94]:
categorical_columns = small_copy.select_dtypes(include=['category'])
#print(categorical_columns.info())
small_3 = pd.concat([small_2,categorical_columns], axis=1)
print(small_3.info())
#print(small_3.head(10))
#small_4 = removenull(small_3, 0.05)
#print(small_4.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 288 entries, Var1 to Var229
dtypes: category(38), float64(125), int64(125)
memory usage: 101.0 MB
None


In [73]:
#START FROM HERE
#We have a dataset, small_3: this contains all 50000 entries, but with some additional added features
#If it's too big a dataset to work with (models take too long to run), let us know - we can do some additional
#cleaning to reduce the size by about half

#print(small_3.info())
#print(small_3.describe())
#print(small_3.head(10))
